In [1]:
!pip install transformers
import transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 6.3 MB/s eta 0:00:00


### Using pre-trained transformers
_for fun and profit_

There are many toolkits that let you access pre-trained transformer models, but the most powerful and convenient by far is [`huggingface/transformers`](https://github.com/huggingface/transformers). In this week's practice, you'll learn how to download, apply and modify pre-trained transformers for a range of tasks. Buckle up, we're going in!


__Pipelines:__ if all you want is to apply a pre-trained model, you can do that in one line of code using pipeline. Huggingface/transformers has a selection of pre-configured pipelines for masked language modelling, sentiment classification, question aswering, etc. ([see full list here](https://huggingface.co/transformers/main_classes/pipelines.html))

A typical pipeline includes:
* pre-processing, e.g. tokenization, subword segmentation
* a backbone model, e.g. bert finetuned for classification
* output post-processing

Let's see it in action:

In [2]:
from transformers import pipeline
classifier = pipeline('sentiment-analysis', model="distilbert-base-uncased-finetuned-sst-2-english")

print(classifier("BERT is amazing!"))

[{'label': 'POSITIVE', 'score': 0.9998860359191895}]


In [3]:
import base64
data = {
    'arryn': 'As High as Honor.',
    'baratheon': 'Ours is the fury.',
    'stark': 'Winter is coming.',
    'tyrell': 'Growing strong.'
}

outputs = {}

# YOUR CODE: predict sentiment for each noble house and create outputs dict
for key in data:
    print(classifier(data[key])[0]['score'] > 0.8)

assert sum(outputs.values()) == 3 and outputs[base64.decodestring(b'YmFyYXRoZW9u\n').decode()] == False
print("Well done!")

True
False
True
True


AssertionError: ignored

You can also access vanilla Masked Language Model that was trained to predict masked words. Here's how:

In [11]:
mlm_model = pipeline('fill-mask', model="bert-base-uncased")
MASK = mlm_model.tokenizer.mask_token

for hypo in mlm_model(f"Donald {MASK} is the president of the united states."):
  print(f"P={hypo['score']:.5f}", hypo['sequence'])

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


P=0.99719 donald trump is the president of the united states.
P=0.00024 donald duck is the president of the united states.
P=0.00022 donald ross is the president of the united states.
P=0.00020 donald johnson is the president of the united states.
P=0.00018 donald wilson is the president of the united states.


In [ ]:
# Your turn: use bert to recall what year was the Soviet Union founded in
mlm_model(<YOUR PROMPT>)

```

```

```

```


Huggingface offers hundreds of pre-trained models that specialize on different tasks. You can quickly find the model you need using [this list](https://huggingface.co/models).


In [14]:
text = """Almost two-thirds of the 1.5 million people who viewed this liveblog had Googled to discover
 the latest on the Rosetta mission. They were treated to this detailed account by the Guardian’s science editor,
 Ian Sample, and astronomy writer Stuart Clark of the moment scientists landed a robotic spacecraft on a comet 
 for the first time in history, and the delirious reaction it provoked at their headquarters in Germany.
  “We are there. We are sitting on the surface. Philae is talking to us,” said one scientist.
"""

# Task: create a pipeline for named entity recognition, use task name 'ner' and search for the right model in the list
ner_model = pipeline('ner', 'dslim/bert-base-NER')
named_entities = ner_model(text)

In [19]:
print('OUTPUT:', named_entities)
word_to_entity = {item['word']: item['entity'] for item in named_entities}
assert 'org' in word_to_entity.get('Guardian').lower() and 'per' in word_to_entity.get('Stuart').lower()
print("All tests passed")

OUTPUT: [{'entity': 'B-LOC', 'score': 0.7991054, 'index': 27, 'word': 'Rose', 'start': 112, 'end': 116}, {'entity': 'I-LOC', 'score': 0.951193, 'index': 28, 'word': '##tta', 'start': 116, 'end': 119}, {'entity': 'B-ORG', 'score': 0.998223, 'index': 40, 'word': 'Guardian', 'start': 179, 'end': 187}, {'entity': 'B-PER', 'score': 0.9997613, 'index': 46, 'word': 'Ian', 'start': 207, 'end': 210}, {'entity': 'I-PER', 'score': 0.99978715, 'index': 47, 'word': 'Sam', 'start': 211, 'end': 214}, {'entity': 'I-PER', 'score': 0.99964595, 'index': 48, 'word': '##ple', 'start': 214, 'end': 217}, {'entity': 'B-PER', 'score': 0.9997831, 'index': 53, 'word': 'Stuart', 'start': 240, 'end': 246}, {'entity': 'I-PER', 'score': 0.9997482, 'index': 54, 'word': 'Clark', 'start': 247, 'end': 252}, {'entity': 'B-LOC', 'score': 0.9997228, 'index': 85, 'word': 'Germany', 'start': 414, 'end': 421}, {'entity': 'B-PER', 'score': 0.9963127, 'index': 99, 'word': 'Phil', 'start': 471, 'end': 475}, {'entity': 'I-PER', '

### The building blocks of a pipeline

Huggingface also allows you to access its pipelines on a lower level. There are two main abstractions for you:
* `Tokenizer` - converts from strings to token ids and back
* `Model` - a pytorch `nn.Module` with pre-trained weights

You can use such models as part of your regular pytorch code: insert is as a layer in your model, apply it to a batch of data, backpropagate, optimize, etc.

In [4]:
import torch
from transformers import AutoTokenizer, AutoModel, pipeline

model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
lines = [
    "Luke, I am your father.",
    "Life is what happens when you're busy making other plans.",
    ]

# tokenize a batch of inputs. "pt" means [p]y[t]orch tensors
tokens_info = tokenizer(lines, padding=True, truncation=True, return_tensors="pt")

for key in tokens_info:
    print(key, tokens_info[key])

print("Detokenized:")
for i in range(2):
    print(tokenizer.decode(tokens_info['input_ids'][i]))

input_ids tensor([[ 101, 5355, 1010, 1045, 2572, 2115, 2269, 1012,  102,    0,    0,    0,
            0,    0,    0],
        [ 101, 2166, 2003, 2054, 6433, 2043, 2017, 1005, 2128, 5697, 2437, 2060,
         3488, 1012,  102]])
token_type_ids tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
attention_mask tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
Detokenized:
[CLS] luke, i am your father. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
[CLS] life is what happens when you're busy making other plans. [SEP]


In [24]:
# You can now apply the model to get embeddings
with torch.no_grad():
    token_embeddings, sentence_embedding = model(**tokens_info)

print(sentence_embedding)

pooler_output


### The search for similar questions.

Remeber week01 where you used GloVe embeddings to find related questions? That was.. cute, but far from state of the art. It's time to **really** solve this task using context-aware embeddings.

In [6]:
# download the data:
!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt
# alternative download link: https://yadi.sk/i/BPQrUu1NaTduEw

--2023-03-19 21:09:15--  https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6031:18::a27d:5112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/obaitrix9jyu84r/quora.txt [following]
--2023-03-19 21:09:15--  https://www.dropbox.com/s/dl/obaitrix9jyu84r/quora.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc161e1b714e92e44632fca0f049.dl.dropboxusercontent.com/cd/0/get/B4kqeDnenpmIC3nD269KxTT7guWU0Wj3x_U4DEbv6UJIP2RAsHR-F2wV7GLg3ZGbsbLZmncpRUDh8uQKSRlsJsABu4RFOt9FERAfg4KOxruQaLrPdaDFvkxTPPOYsNULwVVNnqyZIY_as7Ig1ZueESjMEm-CmeA7ELaQ-Vsn8Q4yfQ/file?dl=1# [following]
--2023-03-19 21:09:15--  https://uc161e1b714e92e44632fca0f049.dl.dropboxusercontent.com/cd/0/get/B4kqeDnenpmIC3nD269KxTT7guWU0Wj3x_U4DEbv6UJIP2RAsHR-F2wV7GLg3ZGbsbLZmncpRUDh8uQKSRls

__Main task(3 pts):__ 
* Implement a function that takes a text string and finds top-k most similar questions from `quora.txt`
* Demonstrate your function using at least 5 examples

There are no prompts this time: you will have to write everything from scratch.

In [7]:
lines = []
with open('quora.txt') as f:
    lines = f.readlines()

lines[:5]

["Can I get back with my ex even though she is pregnant with another guy's baby?\n",
 'What are some ways to overcome a fast food addiction?\n',
 'Who were the great Chinese soldiers and leaders who fought in WW2?\n',
 'What are ZIP codes in the Bay Area?\n',
 'Why was George RR Martin critical of JK Rowling after losing the Hugo award?\n']

In [9]:
# pip install -U sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.7 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=9d13fe033f8b104f334ac1688261218511fee5829c43a249cd25fb7d371fbe08
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence_transformers


In [10]:
from sentence_transformers import SentenceTransformer, util
import torch

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

model = SentenceTransformer('sentence-transformers/LaBSE').to(device)

#Sentences are encoded by calling model.encode()
emb1 = model.encode("This is a red cat with a hat.")
emb2 = model.encode("Have you seen my red cat?")

cos_sim = util.cos_sim(emb1, emb2)
print("Cosine-Similarity:", cos_sim)

Cosine-Similarity: tensor([[0.4557]])


In [30]:
import os

def top_k_similar_questions(model, question, k=5):
    question_embedding = model.encode(question)

    if (os.path.isfile('embeddings.pt')):
      embeddings = torch.load('embeddings.pt')
    else:
      embeddings = model.encode(lines, show_progress_bar=True)
      torch.save(embeddings, 'embeddings.pt')
    
    cos_sims = util.cos_sim(question_embedding, embeddings)
    cos_sims = cos_sims.squeeze()

    all_pairs = []
    for i, cos_sim in enumerate(cos_sims):
      all_pairs.append([cos_sim, i])

    all_pairs_sorted = sorted(all_pairs, reverse=True)

    for pair_cos_sim, i in all_pairs_sorted[:k]:
      print(pair_cos_sim)
      print(lines[i])

In [ ]:
top_k_similar_questions(model, 'Who is Donald Trump?')

In [18]:
import torch
import numpy as np
from transformers import GPT2Tokenizer, GPT2LMHeadModel
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', add_prefix_space=True)
model = GPT2LMHeadModel.from_pretrained('gpt2').train(False).to(device)

text = "The Fermi paradox "
tokens = tokenizer.encode(text)
num_steps = 1024
line_length, max_length = 0, 70

print(end=tokenizer.decode(tokens))

for i in range(num_steps):
    with torch.no_grad():
        logits = model(torch.as_tensor([tokens], device=device))[0]
    p_next = torch.softmax(logits[0, -1, :], dim=-1).data.cpu().numpy()

    next_token_index = np.random.choice(p_next.shape[0], p=p_next)
    # YOUR TASK: change the code so that it performs nucleus sampling

    tokens.append(int(next_token_index))
    print(end=tokenizer.decode(tokens[-1]))
    line_length += len(tokenizer.decode(tokens[-1]))
    if line_length >= max_length:
        line_length = 0
        print()



 The Fermi paradox  becomes apparent; if for some reason he stops saying this impossible thing
, neither man will ever ask himself this contradiction. Why then, should
 the feminine state of mind remain undisturbed, but cause her to lose patience
 till the implication after which she shall be completely oblivious of
 the error or unease ' r.'.  Nature makes art predicated on comb for the
ught and its wits and appliances, and every class in her employ find the
 more use for comb for preservation.  There is only one letter Receipt
 at the end of her number p : X. E1 •X. So that the feminine is no longer
 able to distinguish the first two terms, Hu *[ aaaaeadl, Hte- ntaiaj-
 Caller, aa, thus, MacKenzie's KedsSCVous! Middays' match, say the General
 Grade, is light and neat, he means angles ; but in our observance, illumination
 is sole jive ' for purposes of disguise.' Nor do we notice three times
: a room in which we have selected Microsoft from the light-frame which
 is clearly the abei

RuntimeError: ignored

Transformers knowledge hub: https://huggingface.co/transformers/